In [2]:
import pandas as pd
import re
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
df_onc = pd.read_csv(r'F:\Projects\Pfizer_mCRPC\Data\pre_modelling\EMR_Oncology\Pfizer_mCRPC_ONCEMR_update.csv')
df_uro = pd.read_csv(r'F:\Projects\Pfizer_mCRPC\Data\pre_modelling\EMR_Urology\Pfizer_mCRPC_UROEMR.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (15,16,17,18,70,146,148,150,152,154,156,158,160,162,164,166,174) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_uro.shape
df_onc.shape

(3644, 97)

(5252, 186)

# Clean up data

### Clean up column names

In [5]:
def clean_string(s):
    """
    Function that cleans up string, typically used for column names. Changes
    the incoming string s so that:
        - only lowercase
        - white spaces replaced by underscores
        - brackets replaced by underscores
        - multiple underscores in a row replaced by one
          underscore (e.g. ' (' -> '__' -> '_'
    :param s: string
    :return: new string
    """
    if isinstance(s, float):
        return s
    # only lowercase
    else:
        s = s.lower()
        # whites spaces, brackets, &, ;, , replaced by underscores
        s = re.sub(r'( )|(\()|(\))|(;)|(,)|(&)|(:)', '_', s)
        # multiple underscores in a row replaced by one underscore
        s = re.sub(r'(_)\1+', r'\1', s)
        # trailing underscores are removed
        s = re.sub(r'[_]+$', '', s)
        return s

In [6]:
df_onc.columns = [clean_string(column) for column in df_onc.columns]
df_uro.columns = [clean_string(column) for column in df_uro.columns]

In [7]:
df_onc.head()

,patient_id,lookback_date,index_date,onc_latest_stage,onc_latest_stage_date,met_date1,metac_site_nm1,met_date2,metac_site_nm2,met_date3,...,onc_tnm_date,onc_latest_t,onc_latest_t_date,onc_latest_n,onc_latest_n_date,onc_latest_m,onc_latest_m_date,onc_latest_gleason,onc_latest_gleason_date,pn_flag
0,1563563420,2012-05-14,2014-05-14,Stage IV,2006-11-01,2014-05-14,BONE & CONNECTIVE TISSUE CANCER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1364456765,2015-04-02,2017-04-01,NaN,NaN,2017-04-01,LUNG CANCER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,523105747,2014-09-17,2016-09-16,Stage IV,2012-10-29,2016-09-16,LUNG CANCER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,1151578636,2014-10-04,2016-10-03,Stage IV,2016-10-06,2016-10-03,BONE & CONNECTIVE TISSUE CANCER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,1427814564,2012-03-27,2014-03-27,NaN,NaN,2014-03-27,BONE & CONNECTIVE TISSUE CANCER,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


### Drop metastatic column dates 

In [8]:
ls_metac_colnames = ['met_date1', 'met_date2', 'metac_site_nm2', 'met_date3', 'metac_site_nm3', 'met_date4', 'metac_site_nm4', 'met_date5', 
                     'metac_site_nm5','met_date6', 'metac_site_nm6', 'met_date7', 'metac_site_nm7']

In [9]:
df_onc = df_onc.drop(columns=ls_metac_colnames)
df_uro = df_uro.drop(columns=ls_metac_colnames)

### From meeting 18/01, some more columns to clean up

In [11]:
# delete metastatic date columns
ls_metac_uro = ['deidmetastaticdate', 'deidmetastaticflag']
df_uro = df_uro.drop(columns=ls_metac_uro)

In [31]:
# delete information after the index date for deidcrpc2 and deidcrpc3
df_uro.loc[(df_uro.deidcrpc2date > df_uro.index_date), 'deidcrpc2flag'] = 0
df_uro.loc[(df_uro.deidcrpc2date > df_uro.index_date), 'deidcrpc2date'] = np.nan

df_uro.loc[(df_uro.deidcrpc3date > df_uro.index_date), 'deidcrpc3flag'] = 0
df_uro.loc[(df_uro.deidcrpc3date > df_uro.index_date), 'deidcrpc3date'] = np.nan

In [39]:
# Distribution of latest stages
df_onc.groupby('onc_latest_stage').size()

onc_latest_stage
Stage I        22
Stage II      188
Stage III     103
Stage IV     1717
dtype: int64

In [51]:
# Delete information after the index date
df_onc.loc[df_onc['onc_latest_stage_date'] > df_onc['index_date'], 'onc_latest_stage'] = np.nan
df_onc.loc[df_onc['onc_latest_stage_date'] > df_onc['index_date'], 'onc_latest_stage_date'] = np.nan
df_onc.loc[df_onc['onc_latest_stage_date'] > df_onc['index_date'], 'onc_latest_stage']

Series([], Name: onc_latest_stage, dtype: object)

## Check overlap in columns

In [52]:
uro_colnames = [re.sub('^uro_', '', colname) for colname in df_uro.columns]
df_uro.columns = uro_colnames
onc_colnames = [re.sub('^onc_', '', colname) for colname in df_onc.columns]
df_onc.columns = onc_colnames

In [53]:
# quite good overlap
len(onc_colnames)
len(uro_colnames)
ls_columns_overlap = list(set(uro_colnames) & set(onc_colnames))

173

82

In [54]:
len(ls_columns_overlap)

72

## Check overlap in rows

In [55]:
ss_patientids_overlap = df_uro[df_uro['patient_id'].isin(set(df_onc['patient_id']))]['patient_id']

## Merge the patients that are in both

In [56]:
df_merged_old = df_uro.merge(df_onc, how = 'inner', on = 'patient_id')

In [57]:
df_merged_old.head()

,patient_id,pn_flag_x,lookback_date_x,index_date_x,deidcrpc2flag,deidcrpc2date,deidcrpc3flag,deidcrpc3date,metac_site_nm1_x,psa1_x,...,tnm_date_y,latest_t_y,latest_t_date_y,latest_n_y,latest_n_date_y,latest_m_y,latest_m_date_y,latest_gleason_y,latest_gleason_date_y,pn_flag_y
0,1563563420,0,2012-05-14,2014-05-14,0,NaN,0,NaN,BONE & CONNECTIVE TISSUE CANCER,59.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1317588535,0,2014-04-22,2016-04-21,0,NaN,0,NaN,BONE & CONNECTIVE TISSUE CANCER,NaN,...,2015-06-16,NaN,NaN,N1,2015-06-16,M1,2015-06-16,NaN,NaN,0
2,1303468932,0,2014-09-20,2016-09-19,1,2016-06-22,1,2016-06-22,BONE & CONNECTIVE TISSUE CANCER,14.25,...,2016-02-16,T2,2016-02-16,N0,2016-02-16,M0,2016-02-16,NaN,NaN,0
3,1177110951,0,2012-10-02,2014-10-02,1,2014-02-14,0,NaN,OTHER CANCER,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,495606299,0,2014-04-12,2016-04-11,0,NaN,0,NaN,BONE & CONNECTIVE TISSUE CANCER,879.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


### Merge the columns that are in both for the subset of patients that are in both

#### We can combine psa, total_testo, free_testo and rearrange the dates.

In [58]:
isinstance(ss_patientids_overlap, pd.core.series.Series)
isinstance(ss_patientids_overlap, pd.core.frame.DataFrame)

isinstance(df_merged_old, pd.core.series.Series)
isinstance(df_merged_old, pd.core.frame.DataFrame)

True

False

False

True

In [59]:
def get_stem_date_cols(row, stem):
    if isinstance(row, pd.core.series.Series):
        ls_stem_cols = [column for column in row.index if (re.search(stem, column) and re.search('date', column))]
    else:
        ls_stem_cols = [column for column in row.columns if (re.search(stem, column) and re.search('date', column))]
    return ls_stem_cols

def get_stem_cols(row, stem):
    if isinstance(row, pd.core.series.Series):
        ls_stem_cols = [column for column in row.index if (re.search(stem, column))]
    else:
        ls_stem_cols = [column for column in row.columns if (re.search(stem, column))]
    return ls_stem_cols

In [60]:
def reorder_stem_cols(row_merged, stem):
    ls_date_cols = get_stem_date_cols(row_merged, stem)
    ss_sorted_dates = row_merged[ls_date_cols].sort_values(ascending=False)
    ls_sorted_colnames_dates = ss_sorted_dates.index.tolist()
    ls_sorted_colnames_values = [re.sub('_date', '', date_col) for date_col in ls_sorted_colnames_dates]
    ls_zipped = [item for sublist in zip(ls_sorted_colnames_values, ls_sorted_colnames_dates) for item in sublist]
    return ls_zipped

In [61]:
stems = ['psa', 'total_testo', 'free_testo']

In [62]:
dict_dfs = dict()
for stem in stems:
    ls_cols_stem = get_stem_cols(df_merged_old, stem)
    n_cols_stem = len(ls_cols_stem)
    # create new clean column names
    colnames_values = [stem + '%d' %i for i in range(1, int(n_cols_stem/2) + 1)]
    colnames_dates = [stem + '_date%d' %i for i in range(1, int(n_cols_stem/2) + 1)]
    colnames = [item for sublist in zip(colnames_values, colnames_dates) for item in sublist]
    # Take relevant sub dataframe and make empty one of same shape
    df_unordered = df_merged_old[ls_cols_stem]
    df_reordered = pd.DataFrame(np.nan, index = df_unordered.index, columns=colnames)
    for index, row in df_unordered.iterrows():
        ls_order_colnames = reorder_stem_cols(row, stem)
        df_reordered.iloc[index] = row[ls_order_colnames].tolist()

    df_reordered = df_reordered[df_reordered.isnull().all()[~df_reordered.isnull().all()].index]
    dict_dfs[stem] = df_reordered

In [63]:
df_merged_new = pd.concat(dict_dfs.values(), axis=1)

In [64]:
df_merged_new.shape
df_merged_new.head()

(372, 82)

,psa1,psa_date1,psa2,psa_date2,psa3,psa_date3,psa4,psa_date4,psa5,psa_date5,...,total_testo15,total_testo_date15,total_testo16,total_testo_date16,total_testo17,total_testo_date17,total_testo18,total_testo_date18,free_testo1,free_testo_date1
0,15.60,2014-05-14,55.2,2014-02-19,59.8,2014-01-01,37.80,2013-11-01,36.1,2013-10-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14.25,2016-08-01,10.7,2016-07-01,13.6,2016-06-01,5.55,2016-04-01,5.6,2016-02-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,24.60,2014-06-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,2014-06-03
4,330.80,2016-02-24,879.3,2016-02-01,0.4,2015-07-01,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
df_merged_old.shape
df_merged_old = df_merged_old.drop(columns=get_stem_cols(df_merged_old, 'psa'))
df_merged_old = df_merged_old.drop(columns=get_stem_cols(df_merged_old, 'total_testo'))
df_merged_old = df_merged_old.drop(columns=get_stem_cols(df_merged_old, 'free_testo'))

(372, 254)

In [66]:
df_merged_old.shape
df_merged_old.head()

(372, 94)

,patient_id,pn_flag_x,lookback_date_x,index_date_x,deidcrpc2flag,deidcrpc2date,deidcrpc3flag,deidcrpc3date,metac_site_nm1_x,tnm_date_x,...,tnm_date_y,latest_t_y,latest_t_date_y,latest_n_y,latest_n_date_y,latest_m_y,latest_m_date_y,latest_gleason_y,latest_gleason_date_y,pn_flag_y
0,1563563420,0,2012-05-14,2014-05-14,0,NaN,0,NaN,BONE & CONNECTIVE TISSUE CANCER,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1317588535,0,2014-04-22,2016-04-21,0,NaN,0,NaN,BONE & CONNECTIVE TISSUE CANCER,NaN,...,2015-06-16,NaN,NaN,N1,2015-06-16,M1,2015-06-16,NaN,NaN,0
2,1303468932,0,2014-09-20,2016-09-19,1,2016-06-22,1,2016-06-22,BONE & CONNECTIVE TISSUE CANCER,NaN,...,2016-02-16,T2,2016-02-16,N0,2016-02-16,M0,2016-02-16,NaN,NaN,0
3,1177110951,0,2012-10-02,2014-10-02,1,2014-02-14,0,NaN,OTHER CANCER,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,495606299,0,2014-04-12,2016-04-11,0,NaN,0,NaN,BONE & CONNECTIVE TISSUE CANCER,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


#### tnm stage related columns

In [67]:
ls_latest_stage = [column for column in df_merged_old.columns if 
                   (re.search('latest_', column) and re.search('_(x)|(y)$', column))]
ls_latest_stage.extend(['tnm_date_y', 'tnm_date_x'])
ls_latest_stage.sort(reverse=True)
ls_grouped_colnames = [list(chunck) for chunck in np.array_split(ls_latest_stage, 9)]

In [68]:
ls_grouped_colnames

[['tnm_date_y', 'tnm_date_x'],
 ['latest_t_y', 'latest_t_x'],
 ['latest_t_date_y', 'latest_t_date_x'],
 ['latest_n_y', 'latest_n_x'],
 ['latest_n_date_y', 'latest_n_date_x'],
 ['latest_m_y', 'latest_m_x'],
 ['latest_m_date_y', 'latest_m_date_x'],
 ['latest_gleason_y', 'latest_gleason_x'],
 ['latest_gleason_date_y', 'latest_gleason_date_x']]

In [69]:
df_merged_old_latest = df_merged_old[ls_latest_stage]

In [70]:
# No overlap, latest stage is always only in urology or oncology, never filled in both
for group in ls_grouped_colnames:
    (~df_merged_old_latest[group].isnull()).all(axis=1).any()

False

False

False

False

False

False

False

False

False

In [71]:
def overlay_two_columns(row):
    if row[0] is not np.nan:
        return row[0]
    else:
        return row[1]

In [72]:
df_overlayed = pd.DataFrame()
for group in ls_grouped_colnames:
    colname_no_suffix = re.sub('(_y)|(_x)$', '', group[1])
    df_overlayed[colname_no_suffix] = df_merged_old_latest[group].apply(overlay_two_columns, axis=1)

In [73]:
df_overlayed.head()

,tnm_date,latest_t,latest_t_date,latest_n,latest_n_date,latest_m,latest_m_date,latest_gleason,latest_gleason_date
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-06-16,NaN,NaN,N1,2015-06-16,M1,2015-06-16,NaN,NaN
2,2016-02-16,T2,2016-02-16,N0,2016-02-16,M0,2016-02-16,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
df_merged_new = pd.concat([df_merged_new, df_overlayed], axis=1)
df_merged_old = df_merged_old.drop(columns=ls_latest_stage)

In [75]:
df_merged_old.shape
df_merged_new.shape

(372, 76)

(372, 91)

#### The other columns

In [78]:
ls_leftover = [column for column in df_merged_old.columns if re.search('_[xy]$', column)]
ls_leftover.sort(reverse=True)
ls_leftover
ls_leftover_grouped = [list(chunck) for chunck in np.array_split(ls_leftover, 4)]
ls_leftover_grouped

['pn_flag_y',
 'pn_flag_x',
 'metac_site_nm1_y',
 'metac_site_nm1_x',
 'lookback_date_y',
 'lookback_date_x',
 'index_date_y',
 'index_date_x']

[['pn_flag_y', 'pn_flag_x'],
 ['metac_site_nm1_y', 'metac_site_nm1_x'],
 ['lookback_date_y', 'lookback_date_x'],
 ['index_date_y', 'index_date_x']]

In [79]:
# All these columns are identical in pairs, so data onc and uro is the same, can take one of the two
for group in ls_leftover_grouped:
    (df_merged_old[group[0]] == df_merged_old[group[1]]).all()

True

True

True

True

In [80]:
ls_leftover_y = [column[0] for column in ls_leftover_grouped]
ls_leftover_stem = [re.sub('_y', '', column[0]) for column in ls_leftover_grouped]

In [81]:
df_leftover_y = df_merged_old[ls_leftover_y]
df_leftover_y.columns = ls_leftover_stem
df_leftover_y['patient_id'] = df_merged_old['patient_id']
df_leftover_y.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,pn_flag,metac_site_nm1,lookback_date,index_date,patient_id
0,0,BONE & CONNECTIVE TISSUE CANCER,2012-05-14,2014-05-14,1563563420
1,0,BONE & CONNECTIVE TISSUE CANCER,2014-04-22,2016-04-21,1317588535
2,0,BONE & CONNECTIVE TISSUE CANCER,2014-09-20,2016-09-19,1303468932
3,0,OTHER CANCER,2012-10-02,2014-10-02,1177110951
4,0,BONE & CONNECTIVE TISSUE CANCER,2014-04-12,2016-04-11,495606299


In [82]:
# add to new
df_merged_new = pd.concat([df_leftover_y, df_merged_new], axis=1)

In [83]:
# delete from old
df_merged_old.shape
df_merged_old = df_merged_old.drop(columns = ls_leftover)
df_merged_old.shape

(372, 76)

(372, 68)

### For these patients, add all columns that are only in oncology OR urology

In [84]:
df_merged_total = df_merged_old.merge(df_merged_new, on = 'patient_id', how='outer')

In [85]:
df_merged_total.tail()

,patient_id,deidcrpc2flag,deidcrpc2date,deidcrpc3flag,deidcrpc3date,latest_stage,latest_stage_date,ecog1,ecog_date1,ecog2,...,free_testo_date1,tnm_date,latest_t,latest_t_date,latest_n,latest_n_date,latest_m,latest_m_date,latest_gleason,latest_gleason_date
367,31145451,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
368,200991444,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,...,2015-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
369,57442851,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2012-11-28,T1c,2012-11-28,N0,2012-11-28,M0,2012-11-28,NaN,NaN
370,553351894,1,2015-07-07,1,2016-04-14,Stage IV,2016-09-13,NaN,NaN,NaN,...,NaN,2016-09-13,T1c,2016-09-13,N0,2016-09-13,M1b,2016-09-13,NaN,NaN
371,1627395905,1,2014-11-06,0,NaN,Stage IV,2012-01-01,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Add patients that are only in oncology OR urology 

In [86]:
df_uro_only = df_uro[~df_uro['patient_id'].isin(df_merged_total['patient_id'])]
df_onc_only = df_onc[~df_onc['patient_id'].isin(df_merged_total['patient_id'])]

In [87]:
df_onc_only.shape
df_uro_only.shape

(4880, 173)

(3272, 82)

In [88]:
# These should only contain free_testo, total_testo or psa with higher numbers than in the merged version: CORRECT
set(df_uro_only.columns) - set(df_merged_total.columns);
set(df_onc_only.columns) - set(df_merged_total.columns);

# Also looks ok, only contains columns that we don't expect to have in df_onc or df_uro respectively
set(df_merged_total.columns) - set(df_onc_only.columns);
set(df_merged_total.columns) - set(df_uro_only.columns);

In [89]:
# A total of 186 unique columns
len(set(df_uro_only.columns) | set(df_onc_only.columns) | set(df_merged_total.columns))

183

In [90]:
df_merged_total.shape
df_onc_only.shape
df_uro_only.shape

(372, 163)

(4880, 173)

(3272, 82)

In [91]:
df_onc_and_uro = pd.concat([df_merged_total, df_onc_only, df_uro_only]).reset_index(drop=True)

In [92]:
df_onc_and_uro.tail()

,deidcrpc2date,deidcrpc2flag,deidcrpc3date,deidcrpc3flag,ecog1,ecog10,ecog11,ecog12,ecog13,ecog14,...,total_testo_date17,total_testo_date18,total_testo_date2,total_testo_date3,total_testo_date4,total_testo_date5,total_testo_date6,total_testo_date7,total_testo_date8,total_testo_date9
8519,2014-01-15,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8520,2015-12-03,1.0,2015-12-03,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8521,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8522,2015-05-14,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8523,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [93]:
# sanity checks
df_onc_and_uro['patient_id'].isnull().any()
df_onc_and_uro['pn_flag'].isnull().any()

False

False

In [94]:
# sanity check, did urology/oncology create doubles in columns like psa?
ls_psa = [column for column in df_onc_and_uro.columns if re.search('psa_date', column)]

ss_values = df_onc_and_uro[ls_psa].apply(lambda x: (~x.isnull()).sum(), axis=1)
ss_unique_values = df_onc_and_uro[ls_psa].apply(lambda x: x.nunique(), axis=1)

# seems not
(~(ss_values == ss_unique_values)).sum()

0

## To CSV 

In [95]:
df_onc_and_uro.to_csv(r'F:\Projects\Pfizer_mCRPC\Data\pre_modelling\EMR_combined\Pfizer_mCRPC_EMR_combined.csv', index=False)